# Topology Sort, Longest path in DAG

### Topology Sort Algorithm 1
```python
int current_label = N
for each vertex
    if v not explored
        DFS(G, v)

def DFS(Graph G, vertex v):
    for nxt in G[v]:
        if nxt not explored:
            mark nxt explored
            DFS(G, nxt)
    Label(v) = current_label
    current_label -= 1
    
```
### Topology Sort Algorithm 2  (whenever a node has no pre, add it to order)
Input:   <br>
node <br>
pre <br>
nxt <br>

order = []   <br>
free = node - set(pre)<br>
while free:<br>
    cur = free.pop() <br>
    order.append(cur)<br>
    discard all pre with cur based on nxt<br>
    if no pre, it became a free point, add to free<br>
    

### Longest Path in DAG (need a starting point)
1. Initialize distances to all vertices as minus infinite and distance to source as 0
2. Find a topological sorting of the graph
3. One by one process all vertices in topological order. For every vertex being processed, we update distances of its adjacent using distance of current vertex.
    3.1 Do following for every vertex u in topological order.
            Do following for every adjacent vertex v of u
                if (dist[v] < dist[u] + weight(u, v))
                    dist[v] = dist[u] + weight(u, v)



### Example LC207, 210

There are a total of n courses you have to take, labeled from 0 to n - 1.

Some courses may have prerequisites, for example to take course 0 you have to first take course 1, which is expressed as a pair: [0,1]

Given the total number of courses and a list of prerequisite pairs, return the ordering of courses you should take to finish all courses.

There may be multiple correct orders, you just need to return one of them. If it is impossible to finish all courses, return an empty array.

For example:

> 2, [[1,0]]  <br>
There are a total of 2 courses to take. To take course 1 you should have finished course 0. So the correct course order is [0,1]

> 4, [[1,0],[2,0],[3,1],[3,2]] <br>
There are a total of 4 courses to take. To take course 3 you should have finished both courses 1 and 2. Both courses 1 and 2 should be taken after you finished course 0. So one correct course order is [0,1,2,3]. Another correct ordering is[0,2,1,3].


### Idea
Topology order of courses is the answer

```python
def findOrder(self, numCourses, prerequisites):
        """
        :type numCourses: int
        :type prerequisites: List[List[int]]
        :rtype: List[int]
        """
        graph = {i:[] for i in range(numCourses)}
        for c1, c2 in prerequisites:
                graph[c1].append(c2)

        unvisited = {i:None for i in range(numCourses)}
        ordered_courses = []
        
        def dfs(node):
            del unvisited[node]
            for n in graph[node]:
                if n in unvisited:
                    dfs(n)
            ordered_courses.append(node)
            
        while unvisited:
            dfs(unvisited.keys()[0])
        done = []
        for n in ordered_courses:
            for pre in graph[n]:
                if pre not in done:
                    return []
            done.append(n)
        return done
```


```C++
class Solution {
public:
    vector<int> findOrder(int numCourses, vector<pair<int, int>>& prerequisites) {
        vector<int> result;
        vector<unordered_set<int>> graph(numCourses, unordered_set<int>());
        for(int i=0; i<prerequisites.size(); i++)
            graph[prerequisites[i].second].insert(prerequisites[i].first);
        vector<int> in_degree(numCourses, 0);
        for(int i=0; i<graph.size(); i++)
            for(auto it:graph[i])
                in_degree[it]++;
                
        int count=0;
        for(int i=0; i<numCourses; i++){
            int j;
            for(j=0; j<numCourses; j++) if(in_degree[j]==0)  break;
            /*** return {} means return null vector ***/
            if(j==numCourses)   return {};
            in_degree[j]=-1;
            for(auto it : graph[j])   in_degree[it]--;
            result.push_back(j);
        }
        return result;
    }
};
```


# Longest Path in DAG
### Example2: LC329. Longest Increasing Path in a Matrix
Given an integer matrix, find the length of the longest increasing path.

From each cell, you can either move to four directions: left, right, up or down. You may NOT move diagonally or move outside of the boundary (i.e. wrap-around is not allowed).

Example 1:<br>
<br>
nums = [<br>
  [9,9,4],  <br>
  [6,6,8],<br>
  [2,1,1]<br>
]<br>
Return 4<br>
The longest increasing path is [1, 2, 6, 9].

example2  <br>
[[9, 8, 7], <br>
[2, 1, 6],   <br>
[3, 4, 5]]  <br>
Return 9

**Ideal1: **
DFS and DP
```python
def dfs(i, j):
    if not dp[i][j]:
        val = matrix[i][j]
        dp[i][j] = 1 + max(
        dfs(i - 1, j) if i and val > matrix[i - 1][j] else 0,
        dfs(i + 1, j) if i < M - 1 and val > matrix[i + 1][j] else 0,
        dfs(i, j - 1) if j and val > matrix[i][j - 1] else 0,
        dfs(i, j + 1) if j < N - 1 and val > matrix[i][j + 1] else 0
        )
    return dp[i][j]
```
**Ideal2: **
Treat matrix as graph, smaller node is connected to larger node. Find longest path in graph <br>



In [3]:
def longestIncreasingPath(matrix):
        """
        :type matrix: List[List[int]]
        :rtype: int
        """
        # Topology order part
        if not matrix or not matrix[0]: return 0
        m, n = len(matrix), len(matrix[0])
        visited = []
        order = [[-1]*n for _ in range(m)]
        def dfs(i, j):
            order[i][j] = -2 # in visiting
            for di, dj in [(-1, 0), (1, 0), (0, 1), (0, -1)]:
                if 0<=i+di<m and 0<=j+dj<n and order[i+di][j+dj]==-1 and matrix[i][j] < matrix[i+di][j+dj]:
                    dfs(i+di, j+dj)
            order[i][j] = 1
            visited.append((i, j, matrix[i][j]))
        
        # Longest path part
        [dfs(i, j) for i in range(m) for j in range(n) if order[i][j] == -1]

        c = 1
        visited = visited[::-1]
        order[visited[0][0]][visited[0][1]] = 1
        pre = visited[0][2]
        for i, j, cur in visited:
            for di, dj in [(-1, 0), (1, 0), (0, 1), (0, -1)]:
                if 0<=i+di<m and 0<=j+dj<n and cur < matrix[i+di][j+dj]:
                    order[i+di][j+dj] = max(order[i+di][j+dj], order[i][j]+1)
        return max(map(max, order))

# Example 3, LC 444. Sequence Reconstruction
Check whether the original sequence org can be uniquely reconstructed from the sequences in seqs. The org sequence is a permutation of the integers from 1 to n, with 1 ≤ n ≤ 104. Reconstruction means building a shortest common supersequence of the sequences in seqs (i.e., a shortest sequence so that all sequences in seqs are subsequences of it). Determine whether there is only one sequence that can be reconstructed from seqs and it is the org sequence.

Example 1:

Input:
org: [1,2,3], seqs: [[1,2],[1,3]]

Output:
false

Explanation:
[1,2,3] is not the only one sequence that can be reconstructed, because [1,3,2] is also a valid sequence that can be reconstrcted

### Ideal:
1. Convert seq into graph
2. If the topology order can be uniquely built (only one neighbor once) and equals to org, return True, else False

# Example 4, LC 269. Alien Dictionary
There is a new alien language which uses the latin alphabet. However, the order among letters are unknown to you. You receive a list of non-empty words from the dictionary, where **words are sorted lexicographically by the rules of this new language**. Derive the order of letters in this language.

[
  "wrt",
  "wrf",
  "er",
  "ett",
  "rftt"
]

The correct order is: "wertf"

[
  "z",
  "x",
  "z"
]

The order is invalid, so return "".

### Hard point: build vertical relationship

#### Only the first mismatch letter matters vertically



In [10]:
import collections
def alienOrder(words):
    pre, suc = collections.defaultdict(set), collections.defaultdict(set)
    for pair in zip(words, words[1:]):
        for a, b in zip(*pair):
            if a != b:             # only the first mismatch letter matters
                suc[a].add(b)
                pre[b].add(a)
                break
    print(pre)
    print(suc)
    chars = set(''.join(words))
    free = chars - set(pre)
    order = ''
    while free:
        print('free', free)
        a = free.pop()
        order += a
        for b in suc[a]:
            pre[b].discard(a)
            if not pre[b]:
                free.add(b)
    return order * (set(order) == chars)

words1 = [  "wrtabcde",  "wrf",  "er",  "ett",  "rftt"]
words2 = ["z", "x", "z"]
print(alienOrder(words1))
print('########')
print(alienOrder(words2))

defaultdict(<class 'set'>, {'f': {'t'}, 'e': {'w'}, 't': {'r'}, 'r': {'e'}})
defaultdict(<class 'set'>, {'t': {'f'}, 'w': {'e'}, 'r': {'t'}, 'e': {'r'}})
free {'c', 'a', 'd', 'b', 'w'}
free {'a', 'd', 'b', 'w'}
free {'d', 'b', 'w'}
free {'b', 'w'}
free {'w'}
free {'e'}
free {'r'}
free {'t'}
free {'f'}
cadbwertf
########
defaultdict(<class 'set'>, {'x': {'z'}, 'z': {'x'}})
defaultdict(<class 'set'>, {'z': {'x'}, 'x': {'z'}})

